# My Merrill Edge Portfolio performance analysis 2022 to April 2025

* The result is terrible. We lost over 2M.
* This is because we didn't liquidate ABNB on IPO (2020-12-10). In the 53 months since then, ABNB lagged SP500 by a lot.
  - SP500 growed by 52%
  - ABNB shrank by 17%
  - This single mistake resulted in our big loss


In [ ]:
import yfinance as yf
from curl_cffi import requests
from fmp_data import FMPPriceLoader

pl = FMPPriceLoader()

chrome = requests.Session(impersonate="chrome")
spx = yf.Ticker("^SPX", session=chrome)
spx_2024_2025 = spx.history(start="2018-01-01", end="2025-04-30")

def spx_price(date: str) -> float:
    return spx_2024_2025.loc[date]['Close']



In [76]:
pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Closed.csv")

,Symbol,Acquisition Date,Liquidation Date,Quantity,Acquisition Price ($),Acquisition Cost ($),Liquidation Price ($),Liquidation Amount ($),Gain/Loss ($)
0,TUSK,01/08/2020 (Long Term),06/30/2021,"1,000",$2.23,"$2,228.20",$4.40,"$4,400.00","$2,171.80"
1,TUSK,06/06/2019 (Long Term),06/30/2021,300,$6.09,"$1,827.00",$4.40,"$1,319.99",-$507.01
2,TUSK,06/06/2019 (Long Term),06/30/2021,100,$6.03,$603.00,$4.40,$439.99,-$163.01
3,TUSK,04/22/2019 (Long Term),06/30/2021,85,$17.95,"$1,525.75",$4.40,$373.99,"-$1,151.76"
4,BABA,01/21/2021 (Short Term),08/13/2021,381,$259.65,"$98,927.53",$187.68,"$71,506.63","-$27,420.90"
...,...,...,...,...,...,...,...,...,...
87,ABNB,12/09/2020 (Long Term),02/18/2025,561,144.71,"81,182.31",145,"83,235.22","2,052.91"
88,ABNB,12/09/2020 (Long Term),02/18/2025,439,144.71,"63,527.69",145,"65,134.16","1,606.47"
89,LC,01/17/2025 (Short Term),04/08/2025,"1,800",$16.34,"$29,405.73",$8.90,"$16,019.55","-$13,386.18"
90,CCL,11/18/2024 (Short Term),04/08/2025,590,$24.54,"$14,475.65",$16.95,"$10,003.17","-$4,472.48"


In [77]:
pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Holdings.csv")

,Symbol,Acquisition Date,Quantity,Unit Cost ($),Cost Basis ($),Value($),Gain/Loss ($),Gain/Loss (%)
0,NVDA,04/04/2025 (Short Term),"1,000",103.84,"103,836.53","109,560.00","5,723.47",5.51
1,NVDA,11/29/2024 (Short Term),808,136.7,"110,451.10","88,524.48","-21,926.62",-19.85
2,NVDA,10/22/2024 (Short Term),196,143.15,"28,057.38","21,473.76","-6,583.62",-23.46
3,NVDA,08/21/2024 (Short Term),"4,330",128,"554,240.00","474,394.80","-79,845.20",-14.41
4,NVDA,04/17/2023 (Long Term),830,26.59,"22,073.02","90,934.80","68,861.78",311.97
...,...,...,...,...,...,...,...,...
93,BAC,07/08/2019 (Long Term),99.761,29.28,"2,920.68","3,985.46","1,064.78",36.46
94,JPM,08/14/2019 (Long Term),50,106.66,"5,333.00","12,242.00","6,909.00",129.55
95,JPM,10/31/2019 (Long Term),0.215,125.58,27,52.64,25.64,94.96
96,JPM,07/16/2019 (Long Term),29.785,114.27,"3,403.43","7,292.56","3,889.13",114.27


In [327]:

from fmp_fetch.fmp_online import FMPOnline
from functools import cache

fmp_online = FMPOnline()

@cache
def fmp_price(symbol, date):
    if symbol == 'TWTR':
        return 53.70
    if symbol == 'BRK.B':
        symbol = 'BRK-B'
    return fmp_online.get_price(symbol, date)['adjClose']

@cache
def fmp_pe(symbol, date):
    return fmp_online.get_pe_ratio(symbol, date)

@cache
def fmp_price_to_fcf(symbol, date):
    return fmp_online.get_price_to_fcf(symbol, date)




# Counterfactual Analysis for closed positions: what if instead of buying the stock, we buy the S&P 500 and sell on the liquidation date?

In [332]:
TODAY = '2025-04-29'

closed = pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Closed.csv")

closed['Acquisition Date'] = closed['Acquisition Date'].apply(lambda x: x.replace(' (Long Term)', '').replace(' (Short Term)', ''))
closed['Acquisition Date'] = pd.to_datetime(closed['Acquisition Date']).dt.strftime('%Y-%m-%d')
closed['Liquidation Date'] = pd.to_datetime(closed['Liquidation Date']).dt.strftime('%Y-%m-%d')
closed['Acquisition Cost ($)'] = closed['Acquisition Cost ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
closed['Liquidation Amount ($)'] = closed['Liquidation Amount ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
closed['Liquidation Price ($)'] = closed['Liquidation Price ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
closed['Quantity'] = closed['Quantity'].apply(lambda x: x.replace(',', '')).astype(float)

closed['acquisition_pe'] = closed.apply(lambda x: fmp_pe(x['Symbol'], x['Acquisition Date']), axis=1)
closed['liquidation_pe'] = closed.apply(lambda x: fmp_pe(x['Symbol'], x['Liquidation Date']), axis=1)
closed['acquisition_price_to_fcf'] = closed.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Acquisition Date']), axis=1)
closed['liquidation_price_to_fcf'] = closed.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Liquidation Date']), axis=1)
closed['gain'] = closed['Liquidation Amount ($)'] / closed['Acquisition Cost ($)'] - 1
closed['spx_acquisition_price'] = closed['Acquisition Date'].apply(lambda x: spx_price(x))
closed['spx_liquidation_price'] = closed['Liquidation Date'].apply(lambda x: spx_price(x))
closed['spx_liquidation_counterfactual'] = closed['Acquisition Cost ($)'] / closed['spx_acquisition_price'] * closed['spx_liquidation_price']
closed['minus_spx'] = closed['Liquidation Amount ($)'] - closed['spx_liquidation_counterfactual']
closed['minus_spx_percent'] = closed['minus_spx'] / closed['Acquisition Cost ($)']
closed['spx_gain'] = closed['spx_liquidation_price'] / closed['spx_acquisition_price'] - 1

closed['today'] = TODAY
closed['today_price'] = closed['Symbol'].apply(lambda x: fmp_price(x, TODAY))
closed['today_liquidation_amount'] = closed['Quantity'] * closed['today_price']
closed['today_spx_amount'] = closed['Acquisition Cost ($)'] / closed['spx_acquisition_price'] * spx_price(TODAY)
closed['today_minus_spx'] = closed['today_liquidation_amount'] - closed['today_spx_amount']
closed['sell_early_win'] = closed['minus_spx'] - closed['today_minus_spx']
closed['sell_early_win_percent'] = closed['sell_early_win'] / closed['Acquisition Cost ($)']

2025-05-02 20:38:48 - fmp_online.py:27 - INFO - Querying ABNB at 2020-12-09 which is pre-IPO date will return IPO price ($68)
2025-05-02 20:39:09 - fmp_online.py:52 - INFO - Using PE ratio of 1.0 for VOO
2025-05-02 20:41:18 - fmp_online.py:52 - INFO - Using PE ratio of 1.0 for VOO
2025-05-02 20:42:12 - fmp_online.py:27 - INFO - Querying ABNB at 2020-12-09 which is pre-IPO date will return IPO price ($68)
2025-05-02 20:42:33 - fmp_online.py:80 - INFO - Using price-to-FCF ratio of 1.0 for VOO
2025-05-02 20:43:26 - fmp_online.py:80 - INFO - Using price-to-FCF ratio of 1.0 for VOO


In [333]:
print(f"Compared to S&P 500, our gain is {sum(closed['minus_spx']):,.2f} USD"
    + f", or {sum(closed['minus_spx']) / sum(closed['Acquisition Cost ($)']) * 100:.2f}%\n"
    + f"If holding to {TODAY}, the gain would have been {sum(closed['today_minus_spx']):,.2f} USD")

Compared to S&P 500, our gain is -2,028,932.45 USD, or -54.25%
If holding to 2025-04-29, the gain would have been -1,739,224.68 USD


In [334]:
abnb_closed = closed[closed['Symbol'] == 'ABNB']
non_abnb_closed = closed[closed['Symbol'] != 'ABNB']
print(f"abnb_closed: {abnb_closed.minus_spx.sum():,.2f}, if hold until today: {abnb_closed.today_minus_spx.sum():,.2f}," +
     f" selling gain %: {abnb_closed.sell_early_win.sum() / abnb_closed['Acquisition Cost ($)'].sum() * 100:,.2f}%")
print(f"non_abnb_closed: {non_abnb_closed.minus_spx.sum():,.2f}, {non_abnb_closed.today_minus_spx.sum():,.2f}," +
     f" selling gain %: {non_abnb_closed.sell_early_win.sum() / non_abnb_closed['Acquisition Cost ($)'].sum() * 100:,.2f}%")


abnb_closed: -1,810,863.03, if hold until today: -1,732,270.98, selling gain %: -2.89%
non_abnb_closed: -218,069.42, -6,953.70, selling gain %: -20.71%


# Good deals & bad deals from closed positions

In [335]:
columns = ['Symbol', 'Acquisition Date', 'acquisition_pe', 'acquisition_price_to_fcf', 'Acquisition Cost ($)', 'gain', 'spx_gain', 'minus_spx_percent']
closed.sort_values(by='minus_spx_percent', ascending=True)[columns].head(20)

,Symbol,Acquisition Date,acquisition_pe,acquisition_price_to_fcf,Acquisition Cost ($),gain,spx_gain,minus_spx_percent
3,TUSK,2019-04-22,3.410095,4.131386,1525.75,-0.754881,0.477835,-1.232716
13,T,2019-08-20,6.835151,5.035256,1758.75,-0.304341,0.621970,-0.926311
14,T,2019-08-20,6.835151,5.035256,3516.00,-0.304042,0.621970,-0.926011
71,UPS,2019-09-04,18.678976,18.261873,28.80,0.118056,1.042818,-0.924763
12,T,2019-09-16,7.290082,5.370391,3724.56,-0.343012,0.569247,-0.912259
45,BIDU,2019-05-21,12.198700,45.248061,3088.80,-0.119318,0.769945,-0.889263
70,UPS,2019-12-04,17.573859,24.422383,29.03,0.132277,0.927984,-0.795707
1,TUSK,2019-06-06,1.158825,1.403936,1827.00,-0.277510,0.511347,-0.788857
2,TUSK,2019-06-06,1.158825,1.403936,603.00,-0.270332,0.511347,-0.781679
72,UPS,2019-07-16,16.373070,16.007458,3139.66,0.245358,0.997760,-0.752402


In [336]:
closed.sort_values(by='minus_spx_percent', ascending=False)[columns].head(20)

,Symbol,Acquisition Date,acquisition_pe,acquisition_price_to_fcf,Acquisition Cost ($),gain,spx_gain,minus_spx_percent
65,GOOG,2018-11-26,44.435912,34.028068,1726.26,2.530007,1.174595,1.355412
60,GOOG,2018-11-26,44.435912,34.028068,1726.26,2.530007,1.174595,1.355412
59,GOOG,2018-12-26,44.044257,33.728146,3811.74,2.702349,1.355906,1.346442
64,GOOG,2018-12-26,44.044257,33.728146,3811.74,2.702349,1.355906,1.346442
58,GOOG,2019-01-25,40.237109,32.976823,4444.87,2.365585,1.181686,1.183899
63,GOOG,2019-01-25,40.237109,32.976823,4444.87,2.365585,1.181686,1.183899
57,GOOG,2019-02-25,40.911432,33.529473,4595.82,2.255040,1.079199,1.175841
62,GOOG,2019-02-25,40.911432,33.529473,4595.82,2.255040,1.079199,1.175841
56,GOOG,2019-03-25,43.994054,36.055874,1363.90,1.998182,1.077527,0.920654
61,GOOG,2019-03-25,43.994054,36.055874,1363.90,1.998182,1.077527,0.920654


# Sell V.S Hold

In [337]:
columns = ['Symbol', 'Acquisition Date', 'acquisition_pe','Acquisition Cost ($)', 'liquidation_pe', 'liquidation_price_to_fcf', 'sell_early_win', 'sell_early_win_percent']
non_abnb_closed.sort_values(by='sell_early_win_percent', ascending=True)[columns].head(20)

,Symbol,Acquisition Date,acquisition_pe,Acquisition Cost ($),liquidation_pe,liquidation_price_to_fcf,sell_early_win,sell_early_win_percent
18,CRS,2020-04-15,4.621050,4306.11,-6.285637,9.645697,-35860.521809,-8.327823
36,META,2018-10-16,24.411895,6090.24,16.343714,18.018428,-11765.286430,-1.931826
39,META,2019-06-03,21.541388,5365.82,16.343714,18.018428,-9773.781373,-1.821489
37,META,2018-04-24,29.445193,1991.67,16.343714,18.018428,-3488.086853,-1.751338
27,EDU,2021-08-12,8.255277,69929.81,-2.074597,-1.683357,-112018.193129,-1.601866
26,EDU,2021-08-12,8.255277,4995.96,-2.074597,-1.683357,-8002.389240,-1.601772
34,META,2019-08-05,26.833803,5433.40,16.343714,18.018428,-8690.534423,-1.599465
35,META,2019-08-02,27.909351,3771.00,16.343714,18.018428,-5774.708498,-1.531347
23,EDU,2021-07-26,7.965036,9.70,-2.166944,-1.758289,-12.821896,-1.321845
38,META,2020-06-24,36.135124,29706.90,16.343714,18.018428,-34335.282783,-1.155802


In [338]:
non_abnb_closed.sort_values(by='sell_early_win_percent', ascending=False)[columns].head(20)

,Symbol,Acquisition Date,acquisition_pe,Acquisition Cost ($),liquidation_pe,liquidation_price_to_fcf,sell_early_win,sell_early_win_percent
11,AMZN,2021-01-12,89.673529,31268.10,63.168173,261.051869,41208.561827,1.317911
0,TUSK,2020-01-08,1.925790,2228.20,-5.813266,15.959208,2655.327012,1.191692
2,TUSK,2019-06-06,1.158825,603.00,-5.813266,15.959208,446.896003,0.741121
1,TUSK,2019-06-06,1.158825,1827.00,-5.813266,15.959208,1348.705202,0.738208
8,TWTR,2019-09-19,24.371611,5503.36,135.136733,192.647282,3328.906176,0.604886
5,BABA,2021-01-06,35.661690,11868.75,26.163106,20.697045,6900.593716,0.581409
6,BABA,2020-12-18,30.194658,8849.44,26.163106,20.697045,4946.385484,0.558949
7,BABA,2020-12-10,30.759538,5825.27,26.163106,20.697045,3249.141600,0.557767
4,BABA,2021-01-21,40.736991,98927.53,26.163106,20.697045,54271.749775,0.548601
3,TUSK,2019-04-22,3.410095,1525.75,-5.813266,15.959208,814.982425,0.534152


In [315]:
print(f"Besides ABNB, our gain against SPX counterfactur is {non_abnb_closed.minus_spx.sum():,.2f} US D" +
    f"or {non_abnb_closed.minus_spx.sum() / non_abnb_closed['Acquisition Cost ($)'].sum() * 100:.2f}%")

Besides ABNB, our gain against SPX counterfactur is -218,069.42 US Dor -21.39%


# Counterfactual Analysis for current holdings (2025-04-29): what if we bought S&P 500 instead of the stock?

In [339]:
from datetime import timedelta
current = pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Holdings.csv")

current['Acquisition Date'] = current['Acquisition Date'].apply(lambda x: x.replace(' (Long Term)', '').replace(' (Short Term)', ''))
current['Acquisition Date'] = pd.to_datetime(current['Acquisition Date']).dt.strftime('%Y-%m-%d')
current['Liquidation Date'] = (pd.to_datetime(TODAY) - timedelta(days=1)).strftime('%Y-%m-%d')
current['Cost Basis ($)'] = current['Cost Basis ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
current['Value($)'] = current['Value($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
current['Quantity'] = current['Quantity'].apply(lambda x: x.replace(',', '')).astype(float)


current['acquisition_pe'] = current.apply(lambda x: fmp_pe(x['Symbol'], x['Acquisition Date']), axis=1)
current['liquidation_pe'] = current.apply(lambda x: fmp_pe(x['Symbol'], x['Liquidation Date']), axis=1)
current['acquisition_price_to_fcf'] = current.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Acquisition Date']), axis=1)
current['liquidation_price_to_fcf'] = current.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Liquidation Date']), axis=1)
current['gain'] = current['Value($)'] / current['Cost Basis ($)'] - 1   
current['spx_acquisition_price'] = current['Acquisition Date'].apply(lambda x: spx_price(x))
current['spx_liquidation_price'] = current['Liquidation Date'].apply(lambda x: spx_price(x))
current['spx_liquidation_counterfactual'] = current['Cost Basis ($)'] / current['spx_acquisition_price'] * current['spx_liquidation_price']
current['minus_spx'] = current['Value($)'] - current['spx_liquidation_counterfactual']
current['minus_spx_percent'] = current['minus_spx'] / current['Cost Basis ($)']
current['spx_gain'] = current['spx_liquidation_price'] / current['spx_acquisition_price'] - 1

current.head()

2025-05-02 20:57:37 - fmp_online.py:52 - INFO - Using PE ratio of 1.0 for VOO
2025-05-02 20:57:37 - fmp_online.py:52 - INFO - Using PE ratio of 1.0 for VOO
2025-05-02 20:57:37 - fmp_online.py:52 - INFO - Using PE ratio of 1.0 for VOO
2025-05-02 20:57:37 - fmp_online.py:52 - INFO - Using PE ratio of 1.0 for VOO
2025-05-02 20:58:20 - fmp_online.py:52 - INFO - Using PE ratio of 1.0 for VOO
2025-05-02 20:59:55 - fmp_online.py:80 - INFO - Using price-to-FCF ratio of 1.0 for VOO
2025-05-02 20:59:55 - fmp_online.py:80 - INFO - Using price-to-FCF ratio of 1.0 for VOO
2025-05-02 20:59:55 - fmp_online.py:80 - INFO - Using price-to-FCF ratio of 1.0 for VOO
2025-05-02 20:59:55 - fmp_online.py:80 - INFO - Using price-to-FCF ratio of 1.0 for VOO
2025-05-02 21:00:43 - fmp_online.py:80 - INFO - Using price-to-FCF ratio of 1.0 for VOO


,Symbol,Acquisition Date,Quantity,Unit Cost ($),Cost Basis ($),Value($),Gain/Loss ($),Gain/Loss (%),Liquidation Date,acquisition_pe,liquidation_pe,acquisition_price_to_fcf,liquidation_price_to_fcf,gain,spx_acquisition_price,spx_liquidation_price,spx_liquidation_counterfactual,minus_spx,minus_spx_percent,spx_gain
0,NVDA,2025-04-04,1000.0,103.84,103836.53,109560.00,"5,723.47",5.51,2025-04-28,36.767675,36.622644,41.015593,43.870543,0.055120,5074.080078,5528.75,113140.945038,-3580.945038,-0.034486,0.089606
1,NVDA,2024-11-29,808.0,136.7,110451.10,88524.48,"-21,926.62",-19.85,2025-04-28,64.219057,36.622644,72.759321,43.870543,-0.198519,6032.379883,5528.75,101229.785091,-12705.305091,-0.115031,-0.083488
2,NVDA,2024-10-22,196.0,143.15,28057.38,21473.76,"-6,583.62",-23.46,2025-04-28,83.116301,36.622644,90.006213,43.870543,-0.234648,5851.200195,5528.75,26511.183090,-5037.423090,-0.179540,-0.055108
3,NVDA,2024-08-21,4330.0,128,554240.00,474394.80,"-79,845.20",-14.41,2025-04-28,74.374530,36.622644,80.539794,43.870543,-0.144062,5620.850098,5528.75,545158.534165,-70763.734165,-0.127677,-0.016385
4,NVDA,2023-04-17,830.0,26.59,22073.02,90934.80,"68,861.78",311.97,2025-04-28,113.279493,36.622644,139.878457,43.870543,3.119726,4151.319824,5528.75,29396.966385,61537.833615,2.787921,0.331805


In [340]:
print(f"Compared to S&P 500, our gain is {sum(current['minus_spx'])} USD")
print(f"Compared to S&P 500, our gain excluding ABNB is {sum(current[current.Symbol != 'ABNB']['minus_spx'])} USD")

Compared to S&P 500, our gain is 20013.672794971753 USD
Compared to S&P 500, our gain excluding ABNB is 249584.3195352954 USD


In [341]:
columns = ['Symbol', 'acquisition_pe', 'liquidation_pe', 'acquisition_price_to_fcf', 'liquidation_price_to_fcf', 'Cost Basis ($)', 'Value($)', 'gain', 'spx_acquisition_price', 'spx_liquidation_price', 'spx_gain','minus_spx', 'minus_spx_percent']
current.sort_values(by='minus_spx_percent', ascending=True)[columns].head(20)

,Symbol,acquisition_pe,liquidation_pe,acquisition_price_to_fcf,liquidation_price_to_fcf,Cost Basis ($),Value($),gain,spx_acquisition_price,spx_liquidation_price,spx_gain,minus_spx,minus_spx_percent
61,ABNB,-49.930282,29.386781,-33.720460,17.303511,360183.19,312618.40,-0.132057,3672.820068,5528.75,0.505315,-229570.646740,-0.637372
90,BAC,10.422741,11.516396,22.343183,-36.898183,36.22,40.55,0.119547,3240.020020,5528.75,0.706394,-21.255583,-0.586847
91,BAC,8.924067,11.516396,17.616178,-36.898183,36.00,48.90,0.358333,2961.790039,5528.75,0.866692,-18.300915,-0.508359
93,BAC,8.819569,11.516396,17.409899,-36.898183,2920.68,3985.46,0.364566,2975.949951,5528.75,0.857810,-1440.608924,-0.493244
87,BAC,9.119128,11.516396,18.001231,-36.898183,2964.00,3995.01,0.347844,3006.729980,5528.75,0.838792,-1455.168468,-0.490948
86,BAC,9.119128,11.516396,18.001231,-36.898183,118.52,159.80,0.348296,3006.729980,5528.75,0.838792,-58.133587,-0.490496
92,BAC,8.756871,11.516396,17.286132,-36.898183,2893.61,3995.01,0.380632,3004.040039,5528.75,0.840438,-1330.500339,-0.459806
88,BAC,7.980108,11.516396,15.752796,-36.898183,1326.25,1997.51,0.506134,2840.600098,5528.75,0.946332,-583.812409,-0.440198
89,BAC,7.980108,11.516396,15.752796,-36.898183,5304.80,7990.02,0.506187,2840.600098,5528.75,0.946332,-2334.880370,-0.440145
69,AMZN,89.863174,33.138057,63.146836,59.714979,98176.23,116274.80,0.184348,3795.540039,5528.75,0.456644,-26733.000215,-0.272296


In [342]:
current.sort_values(by='minus_spx_percent', ascending=False)[columns].head(20)

,Symbol,acquisition_pe,liquidation_pe,acquisition_price_to_fcf,liquidation_price_to_fcf,Cost Basis ($),Value($),gain,spx_acquisition_price,spx_liquidation_price,spx_gain,minus_spx,minus_spx_percent
4,NVDA,113.279493,36.622644,139.878457,43.870543,22073.02,90934.80,3.119726,4151.319824,5528.75,0.331805,61537.833615,2.787921
25,AAPL,14.356523,33.264024,13.722397,32.551484,5446.38,25217.45,3.630130,2840.229980,5528.75,0.946585,14615.606617,2.683545
22,AAPL,15.509186,33.264024,14.830133,32.551484,5811.86,25388.40,3.368378,2844.739990,5528.75,0.943499,14093.054525,2.424879
21,AAPL,16.183063,33.264024,15.474505,32.551484,20188.00,84628.00,3.191995,2918.649902,5528.75,0.894283,46386.207139,2.297712
24,AAPL,16.246448,33.264024,15.535114,32.551484,23.10,96.48,3.176623,2847.600098,5528.75,0.941547,51.630260,2.235076
20,AAPL,17.707627,33.264024,16.932318,32.551484,4379.20,16925.60,2.864998,2997.959961,5528.75,0.844171,8849.607553,2.020827
23,AAPL,21.504144,33.264024,20.597266,32.551484,23.19,74.47,2.211298,3096.629883,5528.75,0.785409,33.066372,1.425889
57,GOOG,24.460520,19.978822,32.928681,27.471360,4894.90,14223.16,1.905710,2917.379883,5528.75,0.895108,4946.795775,1.010602
56,GOOG,27.993034,19.978822,30.308126,27.471360,5282.85,15049.71,1.848786,3003.669922,5528.75,0.840665,5325.753075,1.008121
74,MSFT,28.027114,31.347822,29.089914,41.514907,4060.60,11801.10,1.906245,2840.600098,5528.75,0.946332,3897.825523,0.959914
